In [1]:
import numpy as np
import urllib.request
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from tqdm import tqdm
from pathlib import Path

### Get Index of All Weather Stations on NOAA

In [2]:
urllib.request.urlretrieve("https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt",
                           "ghcnd-stations.txt")

with open("ghcnd-stations.txt",'r') as infile:
    data = infile.readlines()

### Check Whether Within Drainage Basin

In [3]:
utah_data = np.array([obj.split()[:4] for obj in data if obj.split()[4]=="UT"])
#GSL_center = (41.1158, -112.4768)
#DRAINAGE BASIN POLYGON (coords eyeballed FROM GOOGLE MAPS) See watershed:
#https://slco.org/watershed/know-your-local-waters/amazing-great-salt-lake/
p1 = (42.285904, -110.697236)
p2 = (40.011010, -111.577385)
p3 = (40.248643, -114.046179)
p4 = (41.991109, -114.056201)

In [4]:
points = [Point(float(x),float(y)) for x,y in utah_data[:,[1,2]]]
polygon = Polygon([p1,p2,p3,p4])
in_basin = np.array([polygon.contains(point) for point in points])
print(in_basin.sum(), "weather stations in the GSL DRAINAGE BASIN")

389 weather stations in the GSL DRAINAGE BASIN


### Download Data

In [5]:
basin_codes = utah_data[in_basin][:,0]
basin_data_urls = list(map(lambda x:"https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/all/"+x+".dly",basin_codes))

In [ ]:
def download_data(loc = "."):
    for i,url in tqdm(list(enumerate(basin_data_urls))):
        urllib.request.urlretrieve(url, Path(loc)/(basin_codes[i]+".dly"))

download_data(".")